# Compute resonance energies and strength in the DR if lithium-like ions

In [1]:
using JAC


The dielectronic recombination of ions is known as as *resonant capture process*, in which an electron is first captured resonantly from the continuum into a doubly-excited (N+1 electron) resonance level and subsequently stabilizes by the emission of photons to some (final N+1 electron) level. Since the excited resonance is embedded into the continuum of the initial ion and with often two or more excited electrons, the first part of a dielectronic recombination is the *inverse* process to an autoionization. The strength of the resonances is depends on the radiative rate, relative to the total decay rate, i.e. the sum of the radiative and re-autoionization rates. For light and medium ions, the excited ion often returns by autoionization back into the inital charge stage, and before a radiative stabilization would occur.

Most often, the dielectronic recombination is described as a two-step (cascade) process in which Auger and radiative (emission) amplitudes and rates need to be combined. To compute these amplitudes, one needs to distinguish between the initial levels (of the N-electron ion) and the intermediate (resonance) and final-state levels (of the N+1 electron ion). The relative level energies of these three multiplets decide, whehther and with which individual strength resonances occur in the spectra, and which correspond to some given electron configurations of the intermediate levels. Often, the pronounced resonances refer to quite high excitations of (at least) one of the electrons.

Here, we shall compute the resonance energies and strength for the dielectronic capture of initially lithium-like ions and, especially, lithlium-like argon $1s^2 2s\quad \to\quad [1s 2s^2 2p + 1s 2s 2p^2]\quad \to\quad 1s^2 2s^2 + 1s^2 2s 2p \quad$. As before, we wish to perform an `Atomic.Computation`, for which we specify the given initial-, intermediate- and final-state configurations. 

Before we do so, let us start from the `Dielectronic.Settings` in order to understand how we control such computations:

In [2]:
? Dielectronic.Settings

`struct  Dielectronic.Settings  <:  AbstractProcessSettings`       ... defines a type for the details and parameters of computing dielectronic recombination pathways.

```
+ multipoles            ::Array{EmMultipoles}  ... Multipoles of the radiation field that are to be included.
+ gauges                ::Array{UseGauge}      ... Specifies the gauges to be included into the computations.
+ calcRateAlpha         ::Bool                 ... True, if the DR rate coefficients are to be calculated, 
                                                   and false o/w.
+ printBefore           ::Bool                 ... True, if all energies and pathways are printed before their 
                                                   evaluation.
+ pathwaySelection      ::PathwaySelection     ... Specifies the selected levels/pathways, if any.
+ electronEnergyShift   ::Float64              ... An overall energy shift for all electron energies (i.e. from 
                                                   the initial to the resonance levels [Hartree].
+ photonEnergyShift     ::Float64              ... An overall energy shift for all photon energies (i.e. from 
                                                   the resonance to the final levels.
+ mimimumPhotonEnergy   ::Float64              ... minimum transition energy for which photon transitions are 
                                                   included into the evaluation.
+ temperatures          ::Array{Float64,1}
+ augerOperator         ::AbstractEeInteraction 
  ... Auger operator that is to be used for evaluating the Auger amplitude's; the allowed values are: 
      CoulombInteraction(), BreitInteration(), CoulombBreit().
```

---

`Dielectronic.Settings()`       ... constructor for the default values of dielectronic recombination pathway computations.

---

`(set::Dielectronic.Settings;`

```
    multipoles=..,           gauges=..,                  calcRateAlpha=..,           printBefore=..,
    pathwaySelection=..,     electronEnergyShift=..,     photonEnergyShift=..,       mimimumPhotonEnergy=..,     
    temperatures=..,         augerOperator=..)
                
... constructor for modifying the given Dielectronic.Settings by 'overwriting' the previously selected parameters.
```


In [3]:
Dielectronic.Settings()

multipoles:                 EmMultipole[E1]  
use-gauges:                 UseGauge[]  
calcRateAlpha:              false  
printBefore:                false  
pathwaySelection:           Inactive PathwaySelection.  
electronEnergyShift:        0.0  
photonEnergyShift:          0.0  
mimimumPhotonEnergy:        0.0  
temperatures:               Float64[]  
augerOperator:              CoulombInteraction()  


From these `Settings`, we see that we can (need to) control both, the `augerOperator` for the capture process as well as the `multipoles` and `gauges` for the radiative stabilization. The meaning of these parameters is completely analogue to the related Auger and photon emission processes in the previous tutorials. In contrast to the (atomic) lines of these processes, however, we now have to deal with (two-step) *DR pathways*, i.e. level triples (initial-intermediate-final). These triples can be specified as before by choosing `selectPathways = true` and by providing a corresponding list `selectedPathways` of such triples. This feature is often helpful to restrict the computations to a few selected pathways. In addition, all the electron and photon energies can be *shifted* can be shifted by a constant amount, a feature that may help to assign the resonances properly. For our example above, we shall take the following settings:

In [4]:
drSettings = Dielectronic.Settings([E1, M1], [JAC.UseCoulomb, JAC.UseBabushkin], false, true, 
                                   PathwaySelection(), 0., 0., 0., Float64[], CoulombInteraction())

multipoles:                 EmMultipole[E1, M1]  
use-gauges:                 UseGauge[UseCoulomb, UseBabushkin]  
calcRateAlpha:              false  
printBefore:                true  
pathwaySelection:           Inactive PathwaySelection.  
electronEnergyShift:        0.0  
photonEnergyShift:          0.0  
mimimumPhotonEnergy:        0.0  
temperatures:               Float64[]  
augerOperator:              CoulombInteraction()  


Apart from the settings, some special care has to be taken for the chosen grid as we need to represent both, the doubly-excited resonances as well as the continuum orbitals, properly. We therefore choose again a logarithmic-linear grid and define the atomic computation as:

In [5]:
grid = Radial.Grid(Radial.Grid(true), rnt = 2.0e-5,h = 5.0e-2, hp = 1.0e-2, rbox = 10.0)

Radial grid:  rnt = 2.0e-5,  h = 0.05,  hp = 0.01,  NoPoints = 1267,  ntL = 194,  ntS = 196, orderL = 7,  orderS = 8,  nsL = 187,  nsS = 188,  mesh = JAC.Radial.MeshGL(), ...  
r:    [1.7802642039531657e-7, 9.041538661464947e-7, 2.0784225157592286e-6]  ...  [10.014443819906385, 10.025962469593956, 10.03308520737538]
wr:   [4.529534173820137e-7, 9.784418738621177e-7, 1.335685766098e-6]  ...  [0.013102024342923369, 0.00959774340257246, 0.004443115926949279]
tS:   [0.0, 0.0, 0.0]  ...  [10.034831506207714, 10.034831506207714, 10.034831506207714]

In [6]:
wa = Atomic.Computation(Atomic.Computation(), name="DR lithium-like argon", grid=grid, nuclearModel=Nuclear.Model(18.), 
                        initialConfigs     =[Configuration("1s^2 2s")],
                        intermediateConfigs=[Configuration("1s 2s^2 2p"), Configuration("1s 2s 2p^2") ],  
                        finalConfigs       =[Configuration("1s^2 2s^2"), Configuration("1s^2 2s 2p")],  
                        processSettings    = drSettings  )
wb = perform(wa)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^1 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 18.0 with mass = 37.62, radius R = 3.37121133160794 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -1.62704536e+02    -1.62704858e+02    +1.97810218e-06    
      2      2s_1/2      -4.07203230e+01    -4.07203638e+01    +1.00312653e-06    
      3      3s_1/2      -1.80782718e+01    -1.80782839e+01    +6.70063399e-07    
      4      4s_1/2      -1.01607481e+01    -1.01607532e+01    +

  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      2p_1/2      -4.07203637e+01    -4.07203638e+01    +3.27566681e-09    
      2      3p_1/2      -1.80782839e+01    -1.80782839e+01    +2.59252292e-09    
      3      4p_1/2      -1.01607532e+01    -1.01607532e+01    +2.05093919e-09    
      4      5p_1/2      -6.49914077e+00    -6.49914079e+00    +1.68334700e-09    
      5      6p_1/2      -4.51139834e+00    -4.51139835e+00    +1.45205668e-09    
      6      7p_1/2      -3.31344405e+00    -3.31344481e+00    +2.28749390e-07    
      7      8p_1/2      -2.53573206e+00    -2.53622787e+00    +1.95531438e-04    
      8      9p_1/2      -1.97861802e+00    -2.00353566e+00    +1.25934605e-02    
      9     10p_1/2      -1.42610726e+00    -1.62260054e+00    +1.37782964e-01    
     10    


Iteration 1 for symmetries ... 
  1s_1/2::  en [a.u.] = -1.3870635e+02;   self-cons'cy = 7.9620e-02  [1.0000e+02 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -2.9792219e+01;   self-cons'cy = 1.5498e-01  [1.0000e+02 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -2.8534601e+01;   self-cons'cy = 1.7596e-01  [1.0000e+02 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -2.8451223e+01;   self-cons'cy = 1.7527e-01  [1.0000e+02 for sym-block kappa = -2]

Iteration 2 for symmetries ... 
  1s_1/2::  en [a.u.] = -1.3972166e+02;   self-cons'cy = 3.6466e-03  [1.0315e-02 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -3.0064445e+01;   self-cons'cy = 4.5480e-03  [1.0315e-02 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -2.8688521e+01;   self-cons'cy = 2.6898e-03  [4.8511e-03 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -2.8566260e+01;   self-cons'cy = 2.0176e-03  [3.5834e-03 for sym-block kappa = -2]

Iteration 3 for symmetries ... 
  1s_1/2::  en [a.u.] = -1.3968230e+02;   self-

  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      2p_1/2      -4.07203637e+01    -4.07203638e+01    +3.27566681e-09    
      2      3p_1/2      -1.80782839e+01    -1.80782839e+01    +2.59252292e-09    
      3      4p_1/2      -1.01607532e+01    -1.01607532e+01    +2.05093919e-09    
      4      5p_1/2      -6.49914077e+00    -6.49914079e+00    +1.68334700e-09    
      5      6p_1/2      -4.51139834e+00    -4.51139835e+00    +1.45205668e-09    
      6      7p_1/2      -3.31344405e+00    -3.31344481e+00    +2.28749390e-07    
      7      8p_1/2      -2.53573206e+00    -2.53622787e+00    +1.95531438e-04    
      8      9p_1/2      -1.97861802e+00    -2.00353566e+00    +1.25934605e-02    
      9     10p_1/2      -1.42610726e+00    -1.62260054e+00    +1.37782964e-01    
     10    

Iteration 1 for symmetries ... 
  1s_1/2::  en [a.u.] = -1.4453636e+02;   self-cons'cy = 5.9133e-02  [1.0000e+02 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -3.0631979e+01;   self-cons'cy = 1.4139e-01  [1.0000e+02 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -2.9544345e+01;   self-cons'cy = 1.5906e-01  [1.0000e+02 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -2.9481717e+01;   self-cons'cy = 1.5797e-01  [1.0000e+02 for sym-block kappa = -2]

Iteration 2 for symmetries ... 
  1s_1/2::  en [a.u.] = -1.4546691e+02;   self-cons'cy = 3.2087e-03  [1.0802e-02 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -3.0929941e+01;   self-cons'cy = 4.8400e-03  [1.0802e-02 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -2.9775708e+01;   self-cons'cy = 3.9003e-03  [7.0934e-03 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -2.9635248e+01;   self-cons'cy = 2.5971e-03  [4.6416e-03 for sym-block kappa = -2]

Iteration 3 for symmetries ... 
  1s_1/2::  en [a.u.] = -1.4542337e+02;   self-c

...

>> (Re-) Define a storage array for various B-spline matrices:
>> Continuum B-spline-Galerkin orbital for energy=8.2462e+01,  kappa=1 [mpt=1067, r[mtp]=8.0692e+00, smallest eigenvalue=2.9957e-13].
>> Pure-sine normalized continuum orbital with normalization constant N=2.4468e+00 and phase phi=2.8489e+00 at r=8.0692e+00 a.u.
Compute (CoulombInteraction()) Auger matrix of dimension 1 x 2 in the continuum- and initial-state bases for the transition [7- ...] and for partial wave p_1/2 ... done. 
>> pathway: 1--7--1 ...
>> (Re-) Define a storage array for various B-spline matrices:
>> Continuum B-spline-Galerkin orbital for energy=8.3091e+01,  kappa=-1 [mpt=1067, r[mtp]=8.0692e+00, smallest eigenvalue=-7.1610e-13].
>> Pure-sine normalized continuum orbital with normalization constant N=2.4488e+00 and phase phi=6.5795e-01 at r=8.0692e+00 a.u.
Compute (CoulombInteraction()) Auger matrix of dimension 1 x 3 in the continuum- and initial-state bases for the transition [8- ...] and for partial wa

>> (Re-) Define a storage array for various B-spline matrices:
>> Continuum B-spline-Galerkin orbital for energy=8.3824e+01,  kappa=2 [mpt=1067, r[mtp]=8.0692e+00, smallest eigenvalue=6.3920e-12].
>> Pure-sine normalized continuum orbital with normalization constant N=2.4191e+00 and phase phi=2.2327e+00 at r=8.0692e+00 a.u.
Compute (CoulombInteraction()) Auger matrix of dimension 1 x 5 in the continuum- and initial-state bases for the transition [16- ...] and for partial wave d_3/2 ... done. 
>> pathway: 1--16--1 ...
>> (Re-) Define a storage array for various B-spline matrices:
>> Continuum B-spline-Galerkin orbital for energy=8.3863e+01,  kappa=-1 [mpt=1067, r[mtp]=8.0692e+00, smallest eigenvalue=-3.7018e-13].
>> Pure-sine normalized continuum orbital with normalization constant N=2.4533e+00 and phase phi=6.4136e-01 at r=8.0692e+00 a.u.
Compute (CoulombInteraction()) Auger matrix of dimension 1 x 6 in the continuum- and initial-state bases for the transition [17- ...] and for partial

      1 --    13 --     5    1/2 + --> 2 + --> 1 -     2.2644e+03   2.2644e+03   3.0623e+03    E1              2.6146e+13     2.1607e+11  2.0594e+11  
      1 --    14 --     1    1/2 + --> 1 + --> 0 +     2.2760e+03   2.2760e+03   3.1262e+03    M1              5.5111e+13     0.0000e+00  0.0000e+00  
      1 --    14 --     2    1/2 + --> 1 + --> 0 -     2.2760e+03   2.2760e+03   3.1033e+03    E1              5.5111e+13     8.1793e+12  8.2841e+12  
      1 --    14 --     3    1/2 + --> 1 + --> 1 -     2.2760e+03   2.2760e+03   3.1023e+03    E1              5.5111e+13     2.9123e+13  2.9761e+13  
      1 --    14 --     4    1/2 + --> 1 + --> 2 -     2.2760e+03   2.2760e+03   3.1001e+03    E1              5.5111e+13     5.7852e+13  5.9526e+13  
      1 --    14 --     5    1/2 + --> 1 + --> 1 -     2.2760e+03   2.2760e+03   3.0740e+03    E1              5.5111e+13     9.0153e+11  8.5589e+11  
      1 --    15 --     1    1/2 + --> 0 + --> 0 +     2.2805e+03   2.2805e+03   3.1306e+03   

      1 --     6 --     4    1/2 + --> 3 + --> 2 -     2.2332e+03   2.2332e+03   3.0574e+03    E1              1.0403e-24     9.1142e-25     
      1 --     6 --     5    1/2 + --> 3 + --> 1 -     2.2332e+03   2.2332e+03   3.0312e+03                    0.0000e+00     0.0000e+00     
      1 --     7 --     1    1/2 + --> 1 - --> 0 +     2.2439e+03   2.2439e+03   3.0941e+03    E1              7.2381e-22     7.3990e-22     
      1 --     7 --     2    1/2 + --> 1 - --> 0 -     2.2439e+03   2.2439e+03   3.0712e+03    M1              9.0942e-29     9.0942e-29     
      1 --     7 --     3    1/2 + --> 1 - --> 1 -     2.2439e+03   2.2439e+03   3.0702e+03    M1              3.2173e-28     3.2173e-28     
      1 --     7 --     4    1/2 + --> 1 - --> 2 -     2.2439e+03   2.2439e+03   3.0680e+03    M1              7.4908e-28     7.4908e-28     
      1 --     7 --     5    1/2 + --> 1 - --> 1 -     2.2439e+03   2.2439e+03   3.0419e+03    M1              9.8522e-30     9.8522e-30     
      

 
  ----------------------------------------------------------------------------------------------------------------------------------------------------------------
       i-level-m           i--J^P--m          Energy        Auger rate     Cou -- rad. rates -- Bab    Cou -- res. strength -- Bab      Widths Gamma_m    
                                               [eV]             [1/s]          [1/s]       [1/s]           [cm^2 eV]   [cm^2 eV]         [eV]            
  ----------------------------------------------------------------------------------------------------------------------------------------------------------------
       1 --    1         1/2 + --> 0 -      2.2274e+03      6.6342e+13      3.2048e+08  3.2048e+08        3.5534e-25  3.5534e-25     4.3667e-02   (4.3667e-02)   
       1 --    2         1/2 + --> 1 -      2.2284e+03      6.4629e+13      2.6604e+12  2.5186e+12        8.4955e-21  8.0598e-21     4.4291e-02   (4.4198e-02)   
       1 --    3         1/2 + --> 1 + 

While the dielectronic recombination of lithium-like ions via the 1s^-1 hole-state resonances is always possible, if free electron with sufficient high energies (~2.2 keV in the example above) are available, such DR processes may occur also at lower energies via excitations into Rydberg orbitals. For the $2s\;\to\; 2p$ excitation of lihtium-like tungsten (Z=74), a dielectronic recombination is possible if electrons are captured into the $7l$ shells for $2s\;\to\; 2p_1/2$ excitations and into the $19l$ shells for $2s\;\to\; 2p_3/2$ excitations, respectively. The modelling of such DR spectra places higher demands on the computations since a proper set of configurations need to be chosen for both, the intermediate- and final-state levels. Here, we do not attempt to perform a detailed computation of this DR spectrum but just indicate how they can still be -- quite readily -- be computed by means of JAC.


In the following, we consider lithlium-like tungsten $1s^2 2s\quad \to\quad [1s^2 2p 7s + 1s^2 2p 7p + 1s^2 2p 7d + 1s^2 2p 7s + 1s^2 2p 7f]\quad \to\quad 1s^2 2s^2 + 1s^2 2s 2p + 1s^2 2s 7s + 1s^2 2s 7p + 1s^2 2s 7d + 1s^2 2s 7f  \quad$. Although the stabilization of the ions via $ 2p\:\to\: 2s$ transitions might be dominant, this simplified computations omits many decay channels due to transitions of the $7l$ electron into the $3l,\; 4l$, etc. levels. For lithlium-like tungsten, moreover, one might expect rather strong relativistic contributions to the binding (transition) energies and the transition amplitudes. Here, we simply omit all these contributions and just compute:

In [7]:
grid = Radial.Grid(Radial.Grid(true), rnt = 2.0e-6,h = 2.0e-2, hp = 5.0e-2, rbox = 10.0)

Radial grid:  rnt = 2.0e-6,  h = 0.02,  hp = 0.05,  NoPoints = 973,  ntL = 152,  ntS = 154, orderL = 7,  orderS = 8,  nsL = 145,  nsS = 146,  mesh = JAC.Radial.MeshGL(), ...  
r:    [6.488575093921812e-9, 3.2953930343170544e-8, 7.575280422116727e-8]  ...  [9.947115048784603, 9.994006561860939, 10.023002664778918]
wr:   [1.650891062239783e-8, 3.566152461805573e-8, 4.8682085366680506e-8]  ...  [0.05333730623526014, 0.03907165531309295, 0.018087573998628273]
tS:   [0.0, 0.0, 0.0]  ...  [10.030111709393754, 10.030111709393754, 10.030111709393754]

In [8]:
wc = Atomic.Computation(Atomic.Computation(), name="DR lithium-like tungsten: 2s -> 2p_1/2", grid=grid, nuclearModel=Nuclear.Model(74.0), 
                        initialConfigs     = [Configuration("1s^2 2s"), Configuration("1s^2 2p"), Configuration("1s^2 7s"), 
                                              Configuration("1s^2 7p"), Configuration("1s^2 7d"), Configuration("1s^2 7f")],
                        intermediateConfigs= [Configuration("1s^2 2s^2"), Configuration("1s^2 2s 2p"), Configuration("1s^2 2p 7s"), 
                                              Configuration("1s^2 2p 7p"), Configuration("1s^2 2p 7d"), Configuration("1s^2 2p 7f")],  
                        finalConfigs       = [Configuration("1s^2 2s^2"), Configuration("1s^2 2s 2p"), Configuration("1s^2 2s 7s"), 
                                              Configuration("1s^2 2s 7p"), Configuration("1s^2 2s 7d"), Configuration("1s^2 2s 7f")],
                        processSettings    = drSettings )

wd = perform(wa)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^1 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 18.0 with mass = 37.62, radius R = 3.37121133160794 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -1.62704536e+02    -1.62704858e+02    +1.97810218e-06    
      2      2s_1/2      -4.07203230e+01    -4.07203638e+01    +1.00312653e-06    
      3      3s_1/2      -1.80782718e+01    -1.80782839e+01    +6.70063399e-07    
      4      4s_1/2      -1.01607481e+01    -1.01607532e+01    +

  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      2p_1/2      -4.07203637e+01    -4.07203638e+01    +3.27566681e-09    
      2      3p_1/2      -1.80782839e+01    -1.80782839e+01    +2.59252292e-09    
      3      4p_1/2      -1.01607532e+01    -1.01607532e+01    +2.05093919e-09    
      4      5p_1/2      -6.49914077e+00    -6.49914079e+00    +1.68334700e-09    
      5      6p_1/2      -4.51139834e+00    -4.51139835e+00    +1.45205668e-09    
      6      7p_1/2      -3.31344405e+00    -3.31344481e+00    +2.28749390e-07    
      7      8p_1/2      -2.53573206e+00    -2.53622787e+00    +1.95531438e-04    
      8      9p_1/2      -1.97861802e+00    -2.00353566e+00    +1.25934605e-02    
      9     10p_1/2      -1.42610726e+00    -1.62260054e+00    +1.37782964e-01    
     10    


Iteration 1 for symmetries ... 
  1s_1/2::  en [a.u.] = -1.3870635e+02;   self-cons'cy = 7.9620e-02  [1.0000e+02 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -2.9792219e+01;   self-cons'cy = 1.5498e-01  [1.0000e+02 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -2.8534601e+01;   self-cons'cy = 1.7596e-01  [1.0000e+02 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -2.8451223e+01;   self-cons'cy = 1.7527e-01  [1.0000e+02 for sym-block kappa = -2]

Iteration 2 for symmetries ... 
  1s_1/2::  en [a.u.] = -1.3972166e+02;   self-cons'cy = 3.6466e-03  [1.0315e-02 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -3.0064445e+01;   self-cons'cy = 4.5480e-03  [1.0315e-02 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -2.8688521e+01;   self-cons'cy = 2.6898e-03  [4.8511e-03 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -2.8566260e+01;   self-cons'cy = 2.0176e-03  [3.5834e-03 for sym-block kappa = -2]

Iteration 3 for symmetries ... 
  1s_1/2::  en [a.u.] = -1.3968230e+02;   self-

  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      2p_1/2      -4.07203637e+01    -4.07203638e+01    +3.27566681e-09    
      2      3p_1/2      -1.80782839e+01    -1.80782839e+01    +2.59252292e-09    
      3      4p_1/2      -1.01607532e+01    -1.01607532e+01    +2.05093919e-09    
      4      5p_1/2      -6.49914077e+00    -6.49914079e+00    +1.68334700e-09    
      5      6p_1/2      -4.51139834e+00    -4.51139835e+00    +1.45205668e-09    
      6      7p_1/2      -3.31344405e+00    -3.31344481e+00    +2.28749390e-07    
      7      8p_1/2      -2.53573206e+00    -2.53622787e+00    +1.95531438e-04    
      8      9p_1/2      -1.97861802e+00    -2.00353566e+00    +1.25934605e-02    
      9     10p_1/2      -1.42610726e+00    -1.62260054e+00    +1.37782964e-01    
     10    

Iteration 1 for symmetries ... 
  1s_1/2::  en [a.u.] = -1.4453636e+02;   self-cons'cy = 5.9133e-02  [1.0000e+02 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -3.0631979e+01;   self-cons'cy = 1.4139e-01  [1.0000e+02 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -2.9544345e+01;   self-cons'cy = 1.5906e-01  [1.0000e+02 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -2.9481717e+01;   self-cons'cy = 1.5797e-01  [1.0000e+02 for sym-block kappa = -2]

Iteration 2 for symmetries ... 
  1s_1/2::  en [a.u.] = -1.4546691e+02;   self-cons'cy = 3.2087e-03  [1.0802e-02 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -3.0929941e+01;   self-cons'cy = 4.8400e-03  [1.0802e-02 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -2.9775708e+01;   self-cons'cy = 3.9003e-03  [7.0934e-03 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -2.9635248e+01;   self-cons'cy = 2.5971e-03  [4.6416e-03 for sym-block kappa = -2]

Iteration 3 for symmetries ... 
  1s_1/2::  en [a.u.] = -1.4542337e+02;   self-c

...

>> (Re-) Define a storage array for various B-spline matrices:
>> Continuum B-spline-Galerkin orbital for energy=8.3115e+01,  kappa=2 [mpt=1067, r[mtp]=8.0692e+00, smallest eigenvalue=5.7110e-12].
>> Pure-sine normalized continuum orbital with normalization constant N=2.4237e+00 and phase phi=2.2517e+00 at r=8.0692e+00 a.u.
Compute (CoulombInteraction()) Auger matrix of dimension 1 x 5 in the continuum- and initial-state bases for the transition [10- ...] and for partial wave d_3/2 ... done. 
>> pathway: 1--10--1 ...
>> (Re-) Define a storage array for various B-spline matrices:
>> Continuum B-spline-Galerkin orbital for energy=8.3138e+01,  kappa=-3 [mpt=1067, r[mtp]=8.0692e+00, smallest eigenvalue=5.1755e-12].
>> Pure-sine normalized continuum orbital with normalization constant N=2.4234e+00 and phase phi=2.2463e+00 at r=8.0692e+00 a.u.
Compute (CoulombInteraction()) Auger matrix of dimension 1 x 2 in the continuum- and initial-state bases for the transition [11- ...] and for partial 

>> (Re-) Define a storage array for various B-spline matrices:
>> Continuum B-spline-Galerkin orbital for energy=8.4322e+01,  kappa=-1 [mpt=1067, r[mtp]=8.0692e+00, smallest eigenvalue=3.7292e-12].
>> Pure-sine normalized continuum orbital with normalization constant N=2.4491e+00 and phase phi=6.3236e-01 at r=8.0692e+00 a.u.
Compute (CoulombInteraction()) Auger matrix of dimension 1 x 6 in the continuum- and initial-state bases for the transition [19- ...] and for partial wave s_1/2 ... done. 
>> (Re-) Define a storage array for various B-spline matrices:
>> Continuum B-spline-Galerkin orbital for energy=8.4322e+01,  kappa=2 [mpt=1067, r[mtp]=8.0692e+00, smallest eigenvalue=3.7385e-12].
>> Pure-sine normalized continuum orbital with normalization constant N=2.4230e+00 and phase phi=2.2187e+00 at r=8.0692e+00 a.u.
Compute (CoulombInteraction()) Auger matrix of dimension 1 x 6 in the continuum- and initial-state bases for the transition [19- ...] and for partial wave d_3/2 ... done. 
>> 

      1 --    13 --     2    1/2 + --> 2 + --> 0 -     2.2644e+03   2.2644e+03   3.0917e+03                    2.6146e+13     0.0000e+00  0.0000e+00  
      1 --    13 --     3    1/2 + --> 2 + --> 1 -     2.2644e+03   2.2644e+03   3.0907e+03    E1              2.6146e+13     1.6549e+13  1.7022e+13  
      1 --    13 --     4    1/2 + --> 2 + --> 2 -     2.2644e+03   2.2644e+03   3.0885e+03    E1              2.6146e+13     2.3971e+14  2.4427e+14  
      1 --    13 --     5    1/2 + --> 2 + --> 1 -     2.2644e+03   2.2644e+03   3.0623e+03    E1              2.6146e+13     2.1607e+11  2.0594e+11  
      1 --    14 --     1    1/2 + --> 1 + --> 0 +     2.2760e+03   2.2760e+03   3.1262e+03    M1              5.5111e+13     0.0000e+00  0.0000e+00  
      1 --    14 --     2    1/2 + --> 1 + --> 0 -     2.2760e+03   2.2760e+03   3.1033e+03    E1              5.5111e+13     8.1793e+12  8.2841e+12  
      1 --    14 --     3    1/2 + --> 1 + --> 1 -     2.2760e+03   2.2760e+03   3.1023e+03   

      1 --     6 --     1    1/2 + --> 3 + --> 0 +     2.2332e+03   2.2332e+03   3.0834e+03                    0.0000e+00     0.0000e+00     
      1 --     6 --     2    1/2 + --> 3 + --> 0 -     2.2332e+03   2.2332e+03   3.0605e+03                    0.0000e+00     0.0000e+00     
      1 --     6 --     3    1/2 + --> 3 + --> 1 -     2.2332e+03   2.2332e+03   3.0596e+03                    0.0000e+00     0.0000e+00     
      1 --     6 --     4    1/2 + --> 3 + --> 2 -     2.2332e+03   2.2332e+03   3.0574e+03    E1              1.0403e-24     9.1142e-25     
      1 --     6 --     5    1/2 + --> 3 + --> 1 -     2.2332e+03   2.2332e+03   3.0312e+03                    0.0000e+00     0.0000e+00     
      1 --     7 --     1    1/2 + --> 1 - --> 0 +     2.2439e+03   2.2439e+03   3.0941e+03    E1              7.2381e-22     7.3990e-22     
      1 --     7 --     2    1/2 + --> 1 - --> 0 -     2.2439e+03   2.2439e+03   3.0712e+03    M1              9.0942e-29     9.0942e-29     
      

      1 --    20 --     4    1/2 + --> 0 + --> 2 -     2.2959e+03   2.2959e+03   3.1201e+03                    0.0000e+00     0.0000e+00     
      1 --    20 --     5    1/2 + --> 0 + --> 1 -     2.2959e+03   2.2959e+03   3.0939e+03    E1              4.0799e-21     4.1757e-21     
  -----------------------------------------------------------------------------------------------------------------------------------------
 
  Total Auger rates, radiative rates and resonance strengths:
 
  ----------------------------------------------------------------------------------------------------------------------------------------------------------------
       i-level-m           i--J^P--m          Energy        Auger rate     Cou -- rad. rates -- Bab    Cou -- res. strength -- Bab      Widths Gamma_m    
                                               [eV]             [1/s]          [1/s]       [1/s]           [cm^2 eV]   [cm^2 eV]         [eV]            
  ------------------------------------